# Import the tools to use

In [1]:
#ignore file warnings
import warnings
warnings.filterwarnings("ignore") 

#natural language preprocessing kit
import nltk
from nltk.stem.lancaster import LancasterStemmer
stemmer = LancasterStemmer()

In [2]:
import numpy as np #for linear algebra
import tflearn as tfl #tensorflow fast to use
import tensorflow as tf #DL model training
import random #to add randomness
import json #dealing with json files
from gtts import gTTS #Google Text-to-Speech
import playsound #to play audio
import os #operating system dealing
import speech_recognition as sr #speech recognition pretraind model

curses is not supported on this machine (please install/reinstall curses for an optimal experience)








In [3]:
#get audio as input to text as output
def get_audio():
    r = sr.Recognizer()
    with sr.Microphone() as sor:
        audio = r.listen(sor)
        what_u_said = ""
        
        try:
            what_u_said = r.recognize_google(audio)
        except:
            speak("speak clearly")
            
    return what_u_said

In [4]:
# move text to API to play as audio
def speak(text):
    tts = gTTS(text=text,lang="en")
    filename = "voice.mp3"
    tts.save(filename)
    playsound.playsound(filename)
    os.remove(filename)

In [5]:
#reading json file
with open("intents.json") as file:
    data = json.load(file)

In [6]:
#text preprocessing
words = []
labels = []
docs_x = []
docs_y = []

for intent in data['intents']:
    for pattern in intent['patterns']:
        wrds = nltk.word_tokenize(pattern)
        words.extend(wrds)
        docs_x.append(wrds)
        docs_y.append(intent["tag"])
        
    if intent['tag'] not in labels:
        labels.append(intent['tag'])

In [7]:
words = [stemmer.stem(w.lower()) for w in words if w != "?"]
words = sorted(list(set(words)))

labels = sorted(labels)

In [8]:
#traing data 

training = []
output = []

out_empty = [0 for _ in range(len(labels))]

for x, doc in enumerate(docs_x):
    bag = []

    wrds = [stemmer.stem(w.lower()) for w in doc]

    for w in words:
        if w in wrds:
            bag.append(1)
        else:
            bag.append(0)

    output_row = out_empty[:]
    output_row[labels.index(docs_y[x])] = 1

    training.append(bag)
    output.append(output_row)

In [9]:
#changing the data to a numpy array
training = np.array(training)
output = np.array(output)

In [10]:
# modelhyperparameters
tf.reset_default_graph()

net = tfl.input_data(shape=[None, len(training[0])])
net = tfl.fully_connected(net, 8)
net = tfl.fully_connected(net, 8)
net = tfl.fully_connected(net, len(output[0]), activation="softmax")
net = tfl.regression(net)

model = tfl.DNN(net)


Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor

Instructions for updating:
keep_dims is deprecated, use keepdims instead

Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where



In [11]:
model.fit(training, output, n_epoch=1000, batch_size=8, show_metric=True)
model.save("model.tflearn")

Training Step: 5999  | total loss: 0.73202 | time: 0.032s
| Adam | epoch: 1000 | loss: 0.73202 - acc: 0.9395 -- iter: 40/44
Training Step: 6000  | total loss: 0.66348 | time: 0.032s
| Adam | epoch: 1000 | loss: 0.66348 - acc: 0.9456 -- iter: 44/44
--
INFO:tensorflow:C:\Users\zd\Desktop\pro\model.tflearn is not in all_model_checkpoint_paths. Manually adding it.


In [12]:
#bag of words
def bag_of_words(s, words):
    bag = [0 for _ in range(len(words))]

    s_words = nltk.word_tokenize(s)
    s_words = [stemmer.stem(word.lower()) for word in s_words]

    for se in s_words:
        for i, w in enumerate(words):
            if w == se:
                bag[i] = 1
            
    return np.array(bag)


In [25]:
def chat():
    print("Start talking with the bot (say quit to stop)!")
    while True:
        print("_____________________________ \nyour turn : ")
        inp = get_audio()
        #inp = input("You: ") #if you want just text to speech
        print("you : ", inp)
        if inp.lower() == "quit":
            break

        results = model.predict([bag_of_words(inp, words)])
        results_index = np.argmax(results)
        tag = labels[results_index]

        for tg in data["intents"]:
            if tg['tag'] == tag:
                responses = tg['responses']
        l=random.choice(responses)
        if l =="":
            speak("speak clearly")
        else:
            speak(l)
            
        print("jessica : ",l)

chat()

Start talking with the bot (say quit to stop)!
_____________________________ 
your turn : 
you :  hi
jessica :  Good to see you again!
_____________________________ 
your turn : 
you :  how are you
jessica :  good thanks for asking
_____________________________ 
your turn : 
you :  tell me a joke
jessica :  Why do fathers take an extra pair of socks when they go golfing?....In case they get a hole in one!
_____________________________ 
your turn : 
you :  tell me another joke
jessica :  How do you get a squirrel to like you? Act like a nut.
_____________________________ 
your turn : 
you :  I like you
jessica :  i love you
_____________________________ 
your turn : 
you :  thank you
jessica :  you are welcome
_____________________________ 
your turn : 
you :  goodbye
jessica :  Sad to see you go :(
_____________________________ 
your turn : 
you :  quit
